# ДЗ Линейная регрессия

В данном задании мы рассмотрим набор данных об учащихся, собранный в 2006 году в одной из школ Португалии. Данные представлены в неудобном для машинного обучения виде, и содержат мусор. Ваша задача &mdash; привести их к надлежащему виду и обучить на них простую модель.

Данные состоят из четырех файлов:
- data.csv &mdash; основная таблица с информацией о учащихся
- scores.csv &mdash; список финальных оценок по одному из предметов (20-балльная шкала переведенная в проценты)
- attendance.csv &mdash; таблица посещений занятий по этому предмету
- school_support.txt &mdash; список учащихся, которым оказывается финансовая поддержка

Ваша задача &mdash; построить модель для предсказания финальных оценок исходя из всех остальных данных и проверить качество ее работы с помощью кросс-валидации. В качестве алгоритма мы будем использовать линейную регрессию

Расшифровка столбцов в data.csv для справки:
- age &mdash; возраст
- Medu &mdash; уровень образования матери (по некоторой условной шкале)
- Fedu &mdash; уровень образования отца (по некоторой условной шкале)
- traveltime &mdash; время в пути до школы (1 – < 15 мин., 2 – от 15 до 30 мин., 3 – от 30 мин. to 1 ч.
или 4 – > 1 ч.)
- studytime &mdash; время, затрачиваемое на занятия вне школы (1 – < 2 ч., 2 – от 2 до 5 ч., 3 – от 5 до 10 ч. или 4 – > 10 ч.)
- famrel &mdash; насколько хорошие отношения в семье у учащегося (по некоторой условной шкале)
- freetime &mdash; количество свободного времени вне школы (по некоторой условной шкале)
- goout &mdash; время, затрачиваемое на общение с друзьями (по некоторой условной шкале)
- Dalc &mdash; количество употребления алкоголя в учебные дни (по некоторой условной шкале)
- Walc &mdash; количество употребления алкоголя в неучебные дни (по некоторой условной шкале)
- health &mdash; уровень здоровья (по некоторой условной шкале)
- sex_M &mdash; пол: мужской (1) или женский (0)
- address_U &mdash; живет ли учащийся в городе (1) или в пригороде (0)
- famsize_LE3 &mdash; размер семьи: не больше 3 человек (1) или больше (0)
- Pstatus_T &mdash; живут ли родители вместе (1) или отдельно (0)
- nursery &mdash; посещал ли учащийся детский сад
- plans_university &mdash; планирует ли учащийся поступать в университет (-1 или 1)
- past_failures &mdash; количество неудовлетворительных оценок по другим предметам ранее (от 0 до 4)

*Примечание. Несколько признаков в данных содержат ошибки/проблемы/некорректности. Эти проблемы нужно исправить. Для
проверки &mdash; всего в данных таких проблем четыре.*

### Задача 1: сломанный признак (а может и не один)
__(1 балл)__

Загрузите таблицу data.csv.

Найдите в данных сломанный признак (он не соответствует описанию) и исправьте его.

In [23]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

data = pd.read_csv('data.csv')

check = []
INF = 1e18
rows = 649

pl = 'plans_university'
pf = 'past_failures'
s = pl + pf

for i in range(rows):
    if data.loc[i, s] < 0:
        check.append(True)
        data.loc[i, s] *= -1
        data.loc[i, s] = str(data.loc[i, s])
        data.loc[i, s] = data.loc[i, s][0] + '*' + data.loc[i, s][1]
    else:
        check.append(False)
        data.loc[i, s] = str(data.loc[i, s])
        data.loc[i, s] = data.loc[i, s][0] + '*' + data.loc[i, s][1]

plpf = data[s].str.split('*', expand=True)

plpf.columns = [pl, pf]

data = pd.concat([data, plpf], axis=1)

data = data.drop(s, axis=1)

for i in range(rows):
    data.loc[i, pf] = int(data.loc[i, pf])
    data.loc[i, pl] = int(data.loc[i, pl])
    if check[i]:
        data.loc[i, pl] *= -1

for i in range(rows):
    if data.loc[i, 'age'] > 100:
        data.loc[i, 'age'] = 2006 - data.loc[i, 'age']

print(data)

C:\Users\Oleg\AppData\Local\Temp\ipykernel_11108\2142278957.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '10' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, s] = str(data.loc[i, s])


     age  Medu  Fedu  traveltime  studytime  famrel  freetime  goout  Dalc  \
0     16     4     4           1          2       5         4    4.0   1.0   
1     17     4     4           1          1       5         3    4.0   1.0   
2     16     1     1           2          1       4         5    5.0   2.0   
3     18     1     2           2          1       3         4    4.0   2.0   
4     17     2     1           2          2       4         2    5.0   1.0   
..   ...   ...   ...         ...        ...     ...       ...    ...   ...   
644   18     2     2           4          2       4         2    5.0   1.0   
645   15     4     4           2          2       4         3    1.0   1.0   
646   21     1     1           2          2       5         3    3.0   5.0   
647   16     2     2           1          1       4         3    4.0   1.0   
648   16     2     3          40          2       4         5    4.0   1.0   

     Walc  health  sex_M  address_U  famsize_LE3  Pstatus_T  nu

### Задача 2: пропуски в данных 
__(1 балл)__

Проверьте, есть ли в данных пропуски (значения NaN). Замените все пропущенные значения на среднее значение этого признака по столбцу.

*Hint: изучите в pandas функции loc, isnull, а также передачу булевых массивов в качестве индексов.*

In [24]:
data = data.fillna(data.mean())

print(data)

     age  Medu  Fedu  traveltime  studytime  famrel  freetime  goout  Dalc  \
0     16     4     4           1          2       5         4    4.0   1.0   
1     17     4     4           1          1       5         3    4.0   1.0   
2     16     1     1           2          1       4         5    5.0   2.0   
3     18     1     2           2          1       3         4    4.0   2.0   
4     17     2     1           2          2       4         2    5.0   1.0   
..   ...   ...   ...         ...        ...     ...       ...    ...   ...   
644   18     2     2           4          2       4         2    5.0   1.0   
645   15     4     4           2          2       4         3    1.0   1.0   
646   21     1     1           2          2       5         3    3.0   5.0   
647   16     2     2           1          1       4         3    4.0   1.0   
648   16     2     3          40          2       4         5    4.0   1.0   

     Walc  health  sex_M  address_U  famsize_LE3  Pstatus_T  nu

### Задача 3: нормализация данных
__(1 балл)__

Нормализуйте данные любым способом

In [25]:
data = (data - data.min()) / (data.max() - data.min())   
print(data)

          age  Medu  Fedu  traveltime  studytime  famrel  freetime  goout  \
0    0.142857  1.00  1.00    0.000000   0.333333    1.00      0.75   0.75   
1    0.285714  1.00  1.00    0.000000   0.000000    1.00      0.50   0.75   
2    0.142857  0.25  0.25    0.020408   0.000000    0.75      1.00   1.00   
3    0.428571  0.25  0.50    0.020408   0.000000    0.50      0.75   0.75   
4    0.285714  0.50  0.25    0.020408   0.333333    0.75      0.25   1.00   
..        ...   ...   ...         ...        ...     ...       ...    ...   
644  0.428571  0.50  0.50    0.061224   0.333333    0.75      0.25   1.00   
645  0.000000  1.00  1.00    0.020408   0.333333    0.75      0.50   0.00   
646  0.857143  0.25  0.25    0.020408   0.333333    1.00      0.50   0.50   
647  0.142857  0.50  0.50    0.000000   0.000000    0.75      0.50   0.75   
648  0.142857  0.50  0.75    0.795918   0.333333    0.75      1.00   0.75   

     Dalc  Walc  health  sex_M  address_U  famsize_LE3  Pstatus_T  nursery 

### Задача 4: кросс-валидация для исходных данных
__(1 балл)__

Загрузите файл scores.csv и протестируйте, как линейная регрессия предсказывает ответ сейчас (с помощью кросс-валидации).

Кроссвалидацию сделайте по 4 разбивкам. Выведите качество в каждом их разбиений.

*Hint: воспользуйтесь sklearn.linear_model и sklearn.model_selection.*

In [26]:
scores = pd.read_csv('scores.csv', header=None)
scores.columns = ['score']
x = data.values
y = scores.values
model = LinearRegression()
res = cross_validate(model, x, y, cv = 4)
res['test_score']

array([0.24687854, 0.26409667, 0.16157011, 0.22620673])

### Задача 5: полные данные
__(2 балла)__

Воспользуйтесь файлами attendance.csv и school_support.txt для того, чтобы добавить новые признаки в данные. Желательно по максимуму использовать возможности pandas для упрощения преобразований.

school_suport число в строке значит что i-ый школьник из исходной таблицы получал мат помощь (обратите внимание что строк в файле меньше, подумайте как правильно импортировать данные)

Добавьте данные таким образом, чтобы качество выросло

In [27]:
attend = pd.read_csv('attendance.csv', delimiter = ";")
support = pd.read_table('school_support.txt', header=None)
support.columns = ['support']
a = []

for j in range(rows):
    cnt = 0
    for i in attend.columns:
        cnt += attend.loc[j, i] == '+'
    a.append(cnt)

data['attend'] = a

a = [0] * rows

for i in range(len(support)):
    a[support.loc[i, 'support']] = 1

data['support'] = a

### Задача 6: борьба с выбросами
__(1.5 балла)__

Качество предсказания может ухудшаться, если в данных присутствуют корректные значения признаков (с точки зрения чтения данных и применения методов), но не соответствующие реальным объектам. Например, данные могли быть введены в неверном формате, а потом слишком грубо приведены к общему виду, из-за чего ошибка не была замечена.
Попробуем от такого избавиться &mdash; а для этого такие объекты нужно сначала найти. Конечно, нам еще недоступны многие продвинутые способы, но давайте попробуем обойтись простыми.

Первый способ это сделать &mdash; посмотреть для каждого признака на распределение его значений и проверить крайние значения на правдоподобность. (постройте гистограммы для признаков, как минимум для подозрительных)

*Hint 1: используйте функцию DataFrame.hist*

*Hint 2: в описании датасета выше есть информация, необходимая для восстановления правильных значений*

In [28]:
s = 'traveltime'

for i in range(rows):
    if data.loc[i, s] > 4:
        if data.loc[i, s] < 15:
            data.loc[i, s] = 1
        elif data.loc[i, s] < 30:
            data.loc[i, s] = 2
        elif data.loc[i, s] < 60:
            data.loc[i, s] = 3
        else:
            data.loc[i, s] = 4

__(1.5 балла)__

Другой простой способ найти выбросы &mdash; сделать предсказание и посчитать ошибку на каждом объекте по отдельности и посмотреть на объекты с наибольшей ошибкой. Обучите линейную регрессию (функция fit) и для каждого объекта посчитайте среднеквадратичное отклонение. Постройте гистограмму распределения ошибок. Посмотрите на гистограмму и удалите из выборки те объекты на которых ошибка слишком большая.

Обратите внимание, что просто удалять все объекты с высокой ошибкой нельзя &mdash; это, конечно, хороший способ добиться меньшей ошибки (на данной выборке), но одновременно вы ухудшите обобщающую способность алгоритма. Вместо этого вам нужно найти однозначно ошибочные записи и их исправить.

*Hint: возможно, все проблемы уже были найдены первым способом; для проверки &mdash; в сумме здесь нужно исправить 3 проблемы.*

Для поиска ошибки на одном отдельном обьекте придётся обучить линейную регрессию руками. Частичный пример, допишите код. Постройте гистограмму распределения ошибок

In [29]:
import sklearn
from sklearn import linear_model
scores = pd.read_csv('scores.csv', header=None)
scores.columns = ['score']
x = data.values
y = scores.values
regression = LinearRegression()
regression = regression.fit(x, y)
error = []
for i in x:
    j = [i]
    prediction = regression.predict(j)
    error.append((prediction[0] - y) ** 2)
array_of_errors = []

for i in error[0]:
    array_of_errors.append(i[0])

array_of_errors.sort()
array_of_errors.reverse()

print(array_of_errors)

[4100.433948727987, 4100.433948727987, 4100.433948727987, 4100.433948727987, 4100.433948727987, 4100.433948727987, 4100.433948727987, 4100.433948727987, 4100.433948727987, 4100.433948727987, 4100.433948727987, 4100.433948727987, 4100.433948727987, 4100.433948727987, 4100.433948727987, 4055.732207129531, 4036.64931787305, 4030.298354787557, 4030.298354787557, 4011.2754655310755, 3485.0876401786218, 1523.7024059811613, 1158.3560974317963, 1158.3560974317963, 1158.3560974317963, 958.8540862900501, 958.8540862900501, 843.0097888824312, 843.0097888824312, 843.0097888824312, 843.0097888824312, 843.0097888824312, 843.0097888824312, 843.0097888824312, 843.0097888824312, 843.0097888824312, 674.2003948394151, 674.2003948394151, 674.2003948394151, 674.2003948394151, 674.2003948394151, 674.2003948394151, 674.2003948394151, 674.2003948394151, 674.2003948394151, 674.2003948394151, 674.2003948394151, 674.2003948394151, 674.2003948394151, 674.2003948394151, 674.2003948394151, 577.6634803330661, 577.66

In [30]:
scores = pd.read_csv('scores.csv', header=None)
scores.columns = ['score']
model = LinearRegression()
res = cross_validate(model, data.values, scores.values, cv = 4)
res['test_score']

array([0.27308425, 0.27895267, 0.13042846, 0.2278476 ])

### Финальное предсказание и отчёт (1 балл)

Проведите предсказание еще раз и сравните качество с исходным. Запишите свои наблюдения - как изменялось качество обучения модели при использовании разных модификаций данных. 

In [31]:
scores = pd.read_csv('scores.csv', header=None)
scores.columns = ['score']
model = LinearRegression()
res = cross_validate(model, data.values, scores.values, cv = 4)
res['test_score']

array([0.27308425, 0.27895267, 0.13042846, 0.2278476 ])

# Report
Худо-бедно написал линрег, качество ужасное, данных мало, ~~расширять не умеем ещё~~, всё грустно в жизни